#### Yu Han, July 2023.
We used logistic regression to classify four cell types for deletion patients only.  Feature importance score is printed. 

mann-whitney-u-test was adopted to run feature by feature analysis between the four cell types to find significant features.

Combining morph features and branching features.

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import statistics
import random
import scipy.stats as ss

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit

import warnings

warnings.filterwarnings("ignore")
from pycytominer import aggregate
import statsmodels.stats.multitest
from parse_cp_features import parse_cp_features

### read profile

In [2]:
# read neuron profile to df
df_neuron = pd.read_csv("profile/neuron_morph_branch_merge.csv")
# groupby Metadata_plate and Metadata_LINE_ID and get mean value of each feature
df_neuron = (
    df_neuron.groupby(["Metadata_LINE_ID_x", "Metadata_Well", "Metadata_GENOTYPE_x"])
    .mean()
    .reset_index()
)
# change Metadata_LINE_ID_x to Metadata_LINE_ID and Metadata_GENOTYPE_x to Metadata_GENOTYPE
df_neuron = df_neuron.rename(
    columns={
        "Metadata_LINE_ID_x": "Metadata_LINE_ID",
        "Metadata_GENOTYPE_x": "Metadata_GENOTYPE",
    }
)
# add column Cell_Type and fill with neuron
df_neuron["Metadata_Cell_Type"] = "neuron"
df_neuron.head(2)

,Metadata_LINE_ID,Metadata_Well,Metadata_GENOTYPE,Metadata_DENSITY_x,Metadata_Site_Count_x,Metadata_Object_Count_x,Cells_AreaShape_Compactness,Cells_AreaShape_MedianRadius,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,...,Nuclei_Texture_SumVariance_Brightfield_20_00,Metadata_DENSITY_y,Metadata_LINE_ID_y,Metadata_Site_Count_y,Metadata_Object_Count_y,Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel,Metadata_Cell_Type
0,1,A01,DELETION,3500.0,3.0,238.0,0.649155,-0.756705,0.402415,-0.242840,...,1.007540,3500.0,1.0,3.0,238.0,-0.766375,-0.666510,-0.131295,-0.886905,neuron
1,1,A02,DELETION,3500.0,3.0,340.0,-0.692455,-0.182364,0.411080,0.634642,...,0.197915,3500.0,1.0,3.0,340.0,-0.920980,-0.749215,-0.420865,-0.990425,neuron


In [3]:
# read stem profile to df
df_stem = pd.read_csv("profile/stem_morph.csv")
# change Metadata_line_ID to Metadata_LINE_ID and Metadata_line_condition to Metadata_GENOTYPE
df_stem = df_stem.rename(
    columns={
        "Metadata_line_ID": "Metadata_LINE_ID",
        "Metadata_line_condition": "Metadata_GENOTYPE",
    }
)
df_stem["Metadata_Cell_Type"] = "stem"
df_stem.head(2)

,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_plating_density,Metadata_LINE_ID,Metadata_GENOTYPE,Metadata_line_source,Cells_AreaShape_Center_X,...,Nuclei_Texture_InfoMeas2_Brightfield_20_00,Nuclei_Texture_InfoMeas2_ER_10_03,Nuclei_Texture_InfoMeas2_RNA_10_03,Nuclei_Texture_InfoMeas2_RNA_20_01,Nuclei_Texture_InverseDifferenceMoment_ER_20_02,Nuclei_Texture_SumAverage_Brightfield_20_03,Nuclei_Texture_SumEntropy_Brightfield_20_01,Nuclei_Texture_SumVariance_DNA_20_03,Nuclei_Texture_SumVariance_Mito_10_00,Metadata_Cell_Type
0,BR_NCP_STEM_1,A01,BR_NCP_STEM_1,BR_NCP_STEM_1,A01,10000,1,control,human,0.50933,...,1.56270,1.1817,0.17543,0.000126,-0.89288,0.630330,0.63990,2.8473,0.82266,stem
1,BR_NCP_STEM_1,A02,BR_NCP_STEM_1,BR_NCP_STEM_1,A02,10000,1,control,human,-0.31877,...,0.65012,1.2649,1.45140,0.280320,-1.23480,-0.062683,-0.26259,2.7774,0.43155,stem


In [4]:
# read progenitor profile to df
df_progen = pd.read_csv("profile/progen_morph_branch_merge.csv")
# change Metadata_line_ID_x to Metadata_LINE_ID and Metadata_line_condition_x to Metadata_GENOTYPE
df_progen = df_progen.rename(
    columns={
        "Metadata_line_ID_x": "Metadata_LINE_ID",
        "Metadata_line_condition_x": "Metadata_GENOTYPE",
    }
)
df_progen["Metadata_Cell_Type"] = "progen"
df_progen.head(2)

,Metadata_Plate,Metadata_Well,Metadata_Site_Count_x,Metadata_Object_Count_x,Metadata_Assay_Plate_Barcode_x,Metadata_Plate_Map_Name_x,Metadata_well_position_x,Metadata_plating_density_x,Metadata_LINE_ID,Metadata_GENOTYPE,...,Metadata_well_position_y,Metadata_plating_density_y,Metadata_line_ID_y,Metadata_line_condition_y,Metadata_line_source_y,Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel,Metadata_Cell_Type
0,BR00127194,A01,9,2486,BR00127194,BR00127194,A01,10000,1,control,...,A01,10000,1,control,human,10.899035,15.247385,4.276348,438.545231,progen
1,BR00127194,A02,9,3014,BR00127194,BR00127194,A02,10000,1,control,...,A02,10000,1,control,human,8.358660,12.026543,3.906105,368.728017,progen


In [5]:
# read astrocyte profile to df
df_astro = pd.read_csv("profile/astro_morph_branch_merge.csv")
# groupby Metadata_plate and Metadata_LINE_ID and get mean value of each feature
df_astro = (
    df_astro.groupby(
        [
            "Metadata_METADATA_SAMPLE_ID_x",
            "Metadata_Well",
            "Metadata_METADATA_CONDITION_x",
        ]
    )
    .mean()
    .reset_index()
)
# change Metadata_METADATA_SAMPLE_ID_x to Metadata_LINE_ID and Metadata_METADATA_CONDITION_x to Metadata_GENOTYPE
df_astro = df_astro.rename(
    columns={
        "Metadata_METADATA_SAMPLE_ID_x": "Metadata_LINE_ID",
        "Metadata_METADATA_CONDITION_x": "Metadata_GENOTYPE",
    }
)
df_astro["Metadata_Cell_Type"] = "astro"
df_astro.head(2)

,Metadata_LINE_ID,Metadata_Well,Metadata_GENOTYPE,Metadata_METADATA_DENSITY_x,Metadata_Site_Count_x,Metadata_Count_Cells_x,Metadata_Count_Cytoplasm_x,Metadata_Count_Nuclei_x,Metadata_Object_Count_x,Image_Granularity_10_Brightfield,...,Metadata_Site_Count_y,Metadata_Count_Cells_y,Metadata_Count_Cytoplasm_y,Metadata_Count_Nuclei_y,Metadata_Object_Count_y,Nuclei_ObjectSkeleton_NumberBranchEnds_CellImageSkel,Nuclei_ObjectSkeleton_NumberNonTrunkBranches_CellImageSkel,Nuclei_ObjectSkeleton_NumberTrunks_CellImageSkel,Nuclei_ObjectSkeleton_TotalObjectSkeletonLength_CellImageSkel,Metadata_Cell_Type
0,1,A01,CONTROL,5000.0,6.5,35.0,35.0,35.0,35.0,-1.131380,...,6.5,35.0,35.0,35.0,35.0,30.0975,29.5485,2.82255,2538.45,astro
1,1,C05,CONTROL,5000.0,7.0,41.5,41.5,41.5,41.5,-1.036655,...,7.0,41.5,41.5,41.5,41.5,67.9125,67.3560,3.27440,4316.00,astro


In [6]:
# concate neuron, stem, progenitor, astrocyte profile to df and keep only common columns
df = pd.concat([df_neuron, df_stem, df_progen, df_astro], axis=0, join="inner")
# change all strings to lower case
df["Metadata_GENOTYPE"] = df["Metadata_GENOTYPE"].str.lower()
df.head(2)

,Metadata_LINE_ID,Metadata_Well,Metadata_GENOTYPE,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_4_0,Cells_AreaShape_Zernike_5_1,Cells_AreaShape_Zernike_6_0,Cells_AreaShape_Zernike_6_2,Cells_AreaShape_Zernike_7_1,...,Nuclei_Texture_Correlation_Brightfield_20_02,Nuclei_Texture_Correlation_DNA_10_01,Nuclei_Texture_Correlation_DNA_10_02,Nuclei_Texture_Correlation_DNA_10_03,Nuclei_Texture_Correlation_ER_20_00,Nuclei_Texture_Correlation_ER_20_01,Nuclei_Texture_Correlation_ER_20_02,Nuclei_Texture_Correlation_Mito_20_00,Nuclei_Texture_Correlation_Mito_20_03,Metadata_Cell_Type
0,1,A01,deletion,-0.242840,0.10070,-0.053765,0.551125,0.269150,0.421625,0.48595,...,0.786645,-1.083280,-0.764545,-0.91867,0.411774,0.047466,0.529805,1.251950,2.022225,neuron
1,1,A02,deletion,0.634642,-0.17572,0.539395,-0.227565,-0.433745,0.067733,-0.09567,...,1.144445,-0.500545,-0.324515,0.07865,-0.155440,-0.397030,-0.084225,1.343547,1.186329,neuron


In [7]:
# select all rows where Metadata_GENOTYPE = deletion from df
df_deletion = df[df["Metadata_GENOTYPE"] == "deletion"]
df_deletion.head(2)

,Metadata_LINE_ID,Metadata_Well,Metadata_GENOTYPE,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_4_0,Cells_AreaShape_Zernike_5_1,Cells_AreaShape_Zernike_6_0,Cells_AreaShape_Zernike_6_2,Cells_AreaShape_Zernike_7_1,...,Nuclei_Texture_Correlation_Brightfield_20_02,Nuclei_Texture_Correlation_DNA_10_01,Nuclei_Texture_Correlation_DNA_10_02,Nuclei_Texture_Correlation_DNA_10_03,Nuclei_Texture_Correlation_ER_20_00,Nuclei_Texture_Correlation_ER_20_01,Nuclei_Texture_Correlation_ER_20_02,Nuclei_Texture_Correlation_Mito_20_00,Nuclei_Texture_Correlation_Mito_20_03,Metadata_Cell_Type
0,1,A01,deletion,-0.242840,0.10070,-0.053765,0.551125,0.269150,0.421625,0.48595,...,0.786645,-1.083280,-0.764545,-0.91867,0.411774,0.047466,0.529805,1.251950,2.022225,neuron
1,1,A02,deletion,0.634642,-0.17572,0.539395,-0.227565,-0.433745,0.067733,-0.09567,...,1.144445,-0.500545,-0.324515,0.07865,-0.155440,-0.397030,-0.084225,1.343547,1.186329,neuron


### logistic regression

In [8]:
# binary code cell types
df_deletion.Metadata_Cell_Type[df_deletion.Metadata_Cell_Type == "neuron"] = 0
df_deletion.Metadata_Cell_Type[df_deletion.Metadata_Cell_Type == "stem"] = 1
df_deletion.Metadata_Cell_Type[df_deletion.Metadata_Cell_Type == "astro"] = 2
df_deletion.Metadata_Cell_Type[df_deletion.Metadata_Cell_Type == "progen"] = 3

In [9]:
# select all feature columns
df_deletion_feat = df_deletion.loc[:, ~df_deletion.columns.str.startswith("Metadata")]

In [10]:
### logistic regression
# define X and y
X = df_deletion_feat.values
y = df_deletion["Metadata_Cell_Type"].tolist()
y0 = np.array(y)

# split training and testing
# using GroupShuffleSplit to ensure the same patient won't appear in train and test set at the same time
gss = GroupShuffleSplit(n_splits=4, train_size=0.8, random_state=42)
gss.get_n_splits()
for train_idx, test_idx in gss.split(X, y0, df_deletion["Metadata_LINE_ID"]):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y0[train_idx], y0[test_idx]

# fit the model
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)
predictions = logisticRegr.predict(X_test)

# print acc score
score = logisticRegr.score(X_test, y_test)
score

0.19117647058823528

### mann-whitney-u-test

In [11]:
# mean profile based on patient ID and genotype
df_deletion_mean = aggregate(
    population_df=df_deletion,
    strata=["Metadata_LINE_ID", "Metadata_Cell_Type"],
    features="infer",
    operation="mean",
)
df_deletion_mean.head(2)

,Metadata_LINE_ID,Metadata_Cell_Type,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_4_0,Cells_AreaShape_Zernike_5_1,Cells_AreaShape_Zernike_6_0,Cells_AreaShape_Zernike_6_2,Cells_AreaShape_Zernike_7_1,Cells_AreaShape_Zernike_7_3,...,Nuclei_Texture_Correlation_Brightfield_20_00,Nuclei_Texture_Correlation_Brightfield_20_02,Nuclei_Texture_Correlation_DNA_10_01,Nuclei_Texture_Correlation_DNA_10_02,Nuclei_Texture_Correlation_DNA_10_03,Nuclei_Texture_Correlation_ER_20_00,Nuclei_Texture_Correlation_ER_20_01,Nuclei_Texture_Correlation_ER_20_02,Nuclei_Texture_Correlation_Mito_20_00,Nuclei_Texture_Correlation_Mito_20_03
0,1,0,0.131600,-0.640063,0.733289,-0.169489,-0.049612,0.686807,-0.304285,-0.230771,...,0.388124,0.283967,-0.284137,-0.507994,-0.038778,0.009464,-0.322690,-0.617292,1.010252,0.921932
1,2,0,0.244952,-0.008189,0.351442,-0.140091,0.283569,-0.035917,0.230835,-0.077175,...,-0.027167,0.431356,-0.457309,-0.443904,-0.524485,0.400303,0.018823,-0.142534,-0.024783,0.067913


In [12]:
# create feature list
feature_list = list(df_deletion_feat.columns)

In [13]:
# create empty dataframe
df_p_values_feature = pd.DataFrame(index=[0], columns=feature_list)

#### Neuron vs. Stem

In [14]:
# run mann-whitney-u test
list_p = []
list_u = []
for feat in feature_list:
    u, p = ss.mannwhitneyu(
        df_deletion_mean.query("Metadata_Cell_Type == 0")[feat],
        df_deletion_mean.query("Metadata_Cell_Type == 1")[feat],
    )
    list_p.append(p)
    list_u.append(u)

In [15]:
list_p_fdr = statsmodels.stats.multitest.fdrcorrection(
    list_p, alpha=0.05, method="indep", is_sorted=False
)

In [16]:
# store p value to each feature
df_p_values_feature.iloc[0] = list_p_fdr[1]
df_p_values_feature

,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_3_1,Cells_AreaShape_Zernike_4_0,Cells_AreaShape_Zernike_5_1,Cells_AreaShape_Zernike_6_0,Cells_AreaShape_Zernike_6_2,Cells_AreaShape_Zernike_7_1,Cells_AreaShape_Zernike_7_3,Cells_AreaShape_Zernike_8_0,Cells_AreaShape_Zernike_8_2,...,Nuclei_Texture_Correlation_Brightfield_20_00,Nuclei_Texture_Correlation_Brightfield_20_02,Nuclei_Texture_Correlation_DNA_10_01,Nuclei_Texture_Correlation_DNA_10_02,Nuclei_Texture_Correlation_DNA_10_03,Nuclei_Texture_Correlation_ER_20_00,Nuclei_Texture_Correlation_ER_20_01,Nuclei_Texture_Correlation_ER_20_02,Nuclei_Texture_Correlation_Mito_20_00,Nuclei_Texture_Correlation_Mito_20_03
0,0.112314,0.112314,0.0988096,0.0988096,0.0947817,0.100157,0.0276418,0.100157,0.0988096,0.100157,...,0.174922,0.116431,0.0988096,0.174922,0.100157,0.128539,0.264534,0.374032,0.363784,0.374032


In [17]:
# select features with p value smaller than .05
df_p_values_feature = df_p_values_feature.drop(
    df_p_values_feature.columns[df_p_values_feature.iloc[-1, :] > 0.05], axis=1
)
sig_features = df_p_values_feature.columns
len(sig_features)

4

In [18]:
# convert df_p_values_feature.columns to a list of string
feature_list = list(df_p_values_feature.columns)

# loop through each string in feature_list and parse it, and add channel to a new list
channel_list = []
for feat in feature_list:
    channel_list.append(parse_cp_features(feat)["channel"])
channel_list

['XNONE', 'DNA_Mito', 'Brightfield', 'Brightfield']

#### Neuron vs. Astrocyte

In [19]:
# run mann-whitney-u test
list_p = []
list_u = []
for feat in feature_list:
    u, p = ss.mannwhitneyu(
        df_deletion_mean.query("Metadata_Cell_Type == 0")[feat],
        df_deletion_mean.query("Metadata_Cell_Type == 2")[feat],
    )
    list_p.append(p)
    list_u.append(u)
list_p_fdr = statsmodels.stats.multitest.fdrcorrection(
    list_p, alpha=0.05, method="indep", is_sorted=False
)

# create empty dataframe
df_p_values_feature = pd.DataFrame(index=[0], columns=feature_list)

# store p value to each feature
df_p_values_feature.iloc[0] = list_p_fdr[1]

# select features with p value smaller than .05
df_p_values_feature = df_p_values_feature.drop(
    df_p_values_feature.columns[df_p_values_feature.iloc[-1, :] > 0.05], axis=1
)
sig_features = df_p_values_feature.columns

# convert df_p_values_feature.columns to a list of string
feature_list = list(df_p_values_feature.columns)

# loop through each string in feature_list and parse it, and add channel to a new list
channel_list = []
for feat in feature_list:
    channel_list.append(parse_cp_features(feat)["channel"])
channel_list

[]

#### Neuron vs. Progenitor

In [20]:
# run mann-whitney-u test
list_p = []
list_u = []
for feat in feature_list:
    u, p = ss.mannwhitneyu(
        df_deletion_mean.query("Metadata_Cell_Type == 0")[feat],
        df_deletion_mean.query("Metadata_Cell_Type == 3")[feat],
    )
    list_p.append(p)
    list_u.append(u)
list_p_fdr = statsmodels.stats.multitest.fdrcorrection(
    list_p, alpha=0.05, method="indep", is_sorted=False
)

# create empty dataframe
df_p_values_feature = pd.DataFrame(index=[0], columns=feature_list)

# store p value to each feature
df_p_values_feature.iloc[0] = list_p_fdr[1]

# select features with p value smaller than .05
df_p_values_feature = df_p_values_feature.drop(
    df_p_values_feature.columns[df_p_values_feature.iloc[-1, :] > 0.05], axis=1
)
sig_features = df_p_values_feature.columns

# convert df_p_values_feature.columns to a list of string
feature_list = list(df_p_values_feature.columns)

# loop through each string in feature_list and parse it, and add channel to a new list
channel_list = []
for feat in feature_list:
    channel_list.append(parse_cp_features(feat)["channel"])
channel_list

[]

#### Stem vs. Astrocyte

In [21]:
# run mann-whitney-u test
list_p = []
list_u = []
for feat in feature_list:
    u, p = ss.mannwhitneyu(
        df_deletion_mean.query("Metadata_Cell_Type == 1")[feat],
        df_deletion_mean.query("Metadata_Cell_Type == 2")[feat],
    )
    list_p.append(p)
    list_u.append(u)
list_p_fdr = statsmodels.stats.multitest.fdrcorrection(
    list_p, alpha=0.05, method="indep", is_sorted=False
)

# create empty dataframe
df_p_values_feature = pd.DataFrame(index=[0], columns=feature_list)

# store p value to each feature
df_p_values_feature.iloc[0] = list_p_fdr[1]

# select features with p value smaller than .05
df_p_values_feature = df_p_values_feature.drop(
    df_p_values_feature.columns[df_p_values_feature.iloc[-1, :] > 0.05], axis=1
)
sig_features = df_p_values_feature.columns

# convert df_p_values_feature.columns to a list of string
feature_list = list(df_p_values_feature.columns)

# loop through each string in feature_list and parse it, and add channel to a new list
channel_list = []
for feat in feature_list:
    channel_list.append(parse_cp_features(feat)["channel"])
channel_list

[]

#### Stem vs. Progenitor

In [22]:
# run mann-whitney-u test
list_p = []
list_u = []
for feat in feature_list:
    u, p = ss.mannwhitneyu(
        df_deletion_mean.query("Metadata_Cell_Type == 1")[feat],
        df_deletion_mean.query("Metadata_Cell_Type == 3")[feat],
    )
    list_p.append(p)
    list_u.append(u)
list_p_fdr = statsmodels.stats.multitest.fdrcorrection(
    list_p, alpha=0.05, method="indep", is_sorted=False
)

# create empty dataframe
df_p_values_feature = pd.DataFrame(index=[0], columns=feature_list)

# store p value to each feature
df_p_values_feature.iloc[0] = list_p_fdr[1]

# select features with p value smaller than .05
df_p_values_feature = df_p_values_feature.drop(
    df_p_values_feature.columns[df_p_values_feature.iloc[-1, :] > 0.05], axis=1
)
sig_features = df_p_values_feature.columns

# convert df_p_values_feature.columns to a list of string
feature_list = list(df_p_values_feature.columns)

# loop through each string in feature_list and parse it, and add channel to a new list
channel_list = []
for feat in feature_list:
    channel_list.append(parse_cp_features(feat)["channel"])
channel_list

[]

#### Astrocyte vs. Progenitor 

In [23]:
# run mann-whitney-u test
list_p = []
list_u = []
for feat in feature_list:
    u, p = ss.mannwhitneyu(
        df_deletion_mean.query("Metadata_Cell_Type == 2")[feat],
        df_deletion_mean.query("Metadata_Cell_Type == 3")[feat],
    )
    list_p.append(p)
    list_u.append(u)
list_p_fdr = statsmodels.stats.multitest.fdrcorrection(
    list_p, alpha=0.05, method="indep", is_sorted=False
)

# create empty dataframe
df_p_values_feature = pd.DataFrame(index=[0], columns=feature_list)

# store p value to each feature
df_p_values_feature.iloc[0] = list_p_fdr[1]

# select features with p value smaller than .05
df_p_values_feature = df_p_values_feature.drop(
    df_p_values_feature.columns[df_p_values_feature.iloc[-1, :] > 0.05], axis=1
)
sig_features = df_p_values_feature.columns

# convert df_p_values_feature.columns to a list of string
feature_list = list(df_p_values_feature.columns)

# loop through each string in feature_list and parse it, and add channel to a new list
channel_list = []
for feat in feature_list:
    channel_list.append(parse_cp_features(feat)["channel"])
channel_list

[]